In [16]:
import numpy as np
import cv2 as cv

# The given video and calibration data
video_file = 'chessboard.avi'
K = np.array([[432.7390364738057, 0, 476.0614994349778],
              [0, 431.2395555913084, 288.7602152621297],
              [0, 0, 1]])
dist_coeff = np.array([-0.2852754904152874, 0.1016466459919075, -0.0004420196146339175, 0.0001149909868437517, -0.01803978785585194])
board_pattern = (10, 7)
board_cellsize = 0.025
board_criteria = cv.CALIB_CB_ADAPTIVE_THRESH + cv.CALIB_CB_NORMALIZE_IMAGE + cv.CALIB_CB_FAST_CHECK

# Open a video
video = cv.VideoCapture(video_file)
assert video.isOpened(), 'Cannot read the given input, ' + video_file

# Prepare a 3D box for simple AR
box_lower = board_cellsize * np.array([[4, 2,  0], [5, 2,  0], [5, 4,  0], [4, 4,  0]])
box_upper = board_cellsize * np.array([[4, 2, -1], [5, 2, -1], [5, 4, -1], [4, 4, -1]])

# Prepare 3D points on a chessboard
obj_points = board_cellsize * np.array([[c, r, 0] for r in range(board_pattern[1]) for c in range(board_pattern[0])])

# Run pose estimation
while True:
    # Read an image from the video
    valid, img = video.read()
    if not valid:
        break

    # Estimate the camera pose
    success, img_points = cv.findChessboardCorners(img, board_pattern, board_criteria)
    if success:
        ret, rvec, tvec = cv.solvePnP(obj_points, img_points, K, dist_coeff)

        # Draw the fixed sphere on the image
        fixed_sphere_center = np.array([0.5, 0.5, 0.5]) * board_cellsize  # Center of the fixed sphere
        fixed_sphere_radius = 0.5 * board_cellsize  # Radius of the fixed sphere
        num_pts = 50  # Number of points to approximate the sphere
        fixed_sphere_points_3d = []
        for theta in np.linspace(0, np.pi, num_pts):
            for phi in np.linspace(0, 2 * np.pi, num_pts):
                x = fixed_sphere_center[0] + fixed_sphere_radius * np.sin(theta) * np.cos(phi)
                y = fixed_sphere_center[1] + fixed_sphere_radius * np.sin(theta) * np.sin(phi)
                z = fixed_sphere_center[2] + fixed_sphere_radius * np.cos(theta)
                fixed_sphere_points_3d.append([x, y, z])
        fixed_sphere_points_3d = np.array(fixed_sphere_points_3d)
        fixed_sphere_points_3d = cv.Rodrigues(np.array([0, np.pi / 2, 0]))[0] @ fixed_sphere_points_3d.T
        fixed_sphere_points_2d, _ = cv.projectPoints(fixed_sphere_points_3d.T, rvec, tvec, K, dist_coeff)
        fixed_sphere_points_2d = np.int32(fixed_sphere_points_2d).reshape(-1, 2)
        
        for i in range(num_pts - 1):
            for j in range(num_pts - 1):
                pt1 = tuple(fixed_sphere_points_2d[i * num_pts + j])
                pt2 = tuple(fixed_sphere_points_2d[i * num_pts + j + 1])
                pt3 = tuple(fixed_sphere_points_2d[(i + 1) * num_pts + j])
                pt4 = tuple(fixed_sphere_points_2d[(i + 1) * num_pts + j + 1])
                cv.line(img, pt1, pt2, (0, 255, 255), 2)
                cv.line(img, pt2, pt4, (0, 255, 255), 2)
                cv.line(img, pt4, pt3, (0, 255, 255), 2)
                cv.line(img, pt3, pt1, (0, 255, 255), 2)

        # Print the camera position
        R, _ = cv.Rodrigues(rvec) # Alternative) `scipy.spatial.transform.Rotation`
        p = (-R.T @ tvec).flatten()
        info = f'XYZ: [{p[0]:.3f} {p[1]:.3f} {p[2]:.3f}]'
        cv.putText(img, info, (10, 25), cv.FONT_HERSHEY_DUPLEX, 0.6, (0, 255, 0))

    # Show the image and process the key event
    cv.imshow('Pose Estimation (Chessboard)', img)
    key = cv.waitKey(10)
    if key == ord(' '):
        key = cv.waitKey()
    if key == 27: # ESC
        break

video.release()
cv.destroyAllWindows()